# How to Use this Template

This template is an example for how to generate a plot using [Bokeh](http://bokeh.pydata.org). This template behaves very similarly to a normal Pinyon/Jupyter template, except that the output consists of the components of a plot. Specifically, the output should be a single variable `pinyon` (yes, overwrite the input data) which has at least the following keys:

+ scripts -> Scripts behind the bokeh plot, generated by bokeh
+ div -> Div for the plot, generated by bokeh

Other keys will be passed to the decision form template along with these variables

For more information about Bokeh plots see [the Bokeh documentation](http://bokeh.pydata.org/en/latest/docs/user_guide/embed.html).

The input data structures contain some special entries for this type of calculation. Namely

+ `pinyon['entry']` contains the entry in question as a Pandas Series object
+ `settings['entry_key']` contains the key for this entry in the larger dataset
+ `settings['decisions']` contains a dictionary of all decisions that have been made about this entry to date. The keys are names of entries, values are a tuple: (old value, new value, notes, ...)

## Input Cell (programmatically generated)

These commands connect to the pinyon server and download the inputs in pkl format into this notebook

In [20]:
import cPickle as pickle
import requests

# Forge some input data
data_pkl = requests.get('http://128.165.58.191:6543/tool/57ab39167af0984c94861cca/data?format=pickle').content
pinyon = {'data':pickle.loads(data_pkl)}
pinyon['entry'] = pinyon['data'].ix[pinyon['data'].index[0]]
settings = dict(entry_key='Bogus',
                decisions=dict(FractionLameller=[None,None,None,dict(crop=range(100,200), mark=range(0,100))]))

## Start Code Here

### Preparatory code
Getting the environment ready (e.g., creating local variables) to make the similarity engine easy

In [45]:
%matplotlib notebook
# Get the data out of the structure
import requests
data = pinyon['data']
entry = pinyon['entry']
target_sample_points = 10000
entry_key = settings['entry_key']

# Get the old decisions
previous_decision = settings['decisions'].get('FractionLameller', [None,None,"",dict(crop=[], mark=[])])
old_selections = previous_decision[3] if len(previous_decision) > 3 else dict(crop=[], mark=[])
old_notes = previous_decision[2]

# Import libraries
from StringIO import StringIO
from skimage import io as skio
import numpy as np
import bokeh.plotting as bp
from math import sqrt, floor

# Ready the Bokeh!
from bokeh.embed import components
from bokeh.layouts import column, widgetbox, row
from bokeh.models import CustomJS, ColumnDataSource, Select, Button, \
    RadioButtonGroup, RadioGroup, Div, LassoSelectTool, TextInput
from bokeh.plotting import Figure, show
from bokeh.io import output_notebook

In [35]:
# Load in BokehJS
output_notebook()

Loading BokehJS ...

### Load in an image
Make a request to the server, get the image

In [36]:
# Load image from URL in dataaset
img = skio.imread(StringIO(requests.get(entry['ImageURL']).content))
img = np.flipud(img)

In [46]:
# Determine the spacing between points
spacing = int(floor(sqrt(np.product(img.shape[:2]) / target_sample_points)))
print "Spacing: ", spacing

Spacing:  2


### Prepare the plot
Create the datasets that hold image and markup data

In [43]:
# Convert image to grayscale
img_1d = np.mean(img, axis=2)

# Make points for each spot on the graph
xx, yy = np.meshgrid(range(0,img.shape[0],spacing), range(0,img.shape[1],spacing))
xx = xx.ravel()
yy = yy.ravel()

bokeh_data = ColumnDataSource(data={'x': xx, 'y': yy})
img_data = ColumnDataSource(data={'img':[img_1d], 'width':[img.shape[0]], 'height':[img.shape[1]]})

Prepare the plot

In [6]:
plot = Figure(plot_width=800, 
                 x_range=[0, img.shape[0]], y_range=[0, img.shape[1]],
                 tools="pan,wheel_zoom,box_zoom,reset,resize", webgl=True)
plot.add_tools(LassoSelectTool(select_every_mousemove=False))
plot.image(image='img', x=[0], y=[0], dw='width', dh='height', source=img_data)

Create plots that will display marked points

In [7]:
# Datasets to hold cropped and marked points
crop_points = ColumnDataSource(data={'x':xx[old_selections['crop']],
                                     'y':yy[old_selections['crop']],
                                     'ind':old_selections['crop']})
mark_points = ColumnDataSource(data={'x':xx[old_selections['mark']],
                                     'y':yy[old_selections['mark']],
                                     'ind':old_selections['mark']})

In [8]:
# Hidden list of points behind the plot
plot.scatter('x', 'y', visible=False, source=bokeh_data)
plot.rect(x='x', y='y', width=spacing*0.5, height=spacing*0.5, source=crop_points, color='red', alpha=0.5)
plot.rect(x='x', y='y', width=spacing*0.5, height=spacing*0.5, source=mark_points, color='blue', alpha=0.5)

### Prepare the controls

Create controls for the painters

In [16]:
mark_type = Select(options=['Crop', 'Mark'], value='Crop')
brush_type = Select(options=['Add', 'Remove'], value='Add')
submit_button = Button(label='Submit', button_type='success')
compute_button = Button(label='Compute')
results_box = Div(width=200, height=20)
notes_box = TextInput(value=old_notes, title="Notes:", width=800)

Create callback for marking points

In [10]:
bokeh_data.callback = CustomJS(
    args=dict(cropped=crop_points, marked=mark_points, mark_box=mark_type, brush_box=brush_type),
    code="""
    var inds = cb_obj.get('selected')['1d'].indices;
    var x_points = cb_obj.get('data')['x'];
    var y_points = cb_obj.get('data')['y'];
    var crop_data = cropped.get('data');
    var mark_data = marked.get('data');
    var mark_type = mark_box.get('value');
    var brush_type = brush_box.get('value');
    
    if (inds.length == 0) { return; }
    
    // Get the appropriate data
    var active_data;
    if (mark_type === 'Crop') {
        active_data = crop_data;
    } else {
        active_data = mark_data;
    }
    
    // Get the current IDs
    var new_inds = new Set();
    for (i of active_data['ind']) {
        new_inds.add(i);
    }
    
    // Add or remove brush
    if (brush_type === 'Add') {
        for (i of inds) {
            new_inds.add(i);
        }
    } else {
        for (i of inds) {
            new_inds.delete(i);
        }
    }

    // Add data to cropped
    active_data['x'] = [];
    active_data['y'] = [];
    active_data['ind'] = [v for (v of new_inds)];
    for (i of active_data['ind']) {
        active_data['x'].push(x_points[i]);
        active_data['y'].push(y_points[i]);
    }

    // Update the plot
    cropped.trigger('change');
    marked.trigger('change');
""")

In [11]:
button_callback = CustomJS(
    args=dict(cropped=crop_points, marked=mark_points, all_data=bokeh_data,
              output_frame=results_box, notes_box=notes_box), 
    code="""
    var cb_name = cb_obj.get('label');
    var crop_data = cropped.get('data');
    var mark_data = marked.get('data');
    var all_data = all_data.get('data');
    
    // Get the ids that were not cropped
    var cropped_ids = new Set([x for (x of crop_data['ind'])]);
    var all_ids = new Set();
    for (var i=0; i<all_data['x'].length; i++) {
        all_ids.add(i);
    }
    var valid_ids = new Set([...all_ids].filter(x => !cropped_ids.has(x)));
    
    // Of those, get the ones that were marked
    var marked_ids = new Set([x for (x of mark_data['ind'])]);
    var marked_valid_ids = new Set([...valid_ids].filter(x => marked_ids.has(x)));
    
    // Compute the fraction
    var frac = marked_valid_ids.size / valid_ids.size;
    output_frame.set('text', 'Marked Fraction: ' 
            + new Intl.NumberFormat("en-US", {minimumFractionDigits: 4}).format(frac));
            
    // Run submission
    if (cb_name === "Submit") {
        var output = {};
        var key = JSON.stringify(['%s','%s']);
        var selection_record = {};
        selection_record['crop'] = crop_data['ind'];
        selection_record['mark'] = mark_data['ind'];
        output[key] = ['%s', frac, notes_box.get('value'), selection_record];
        
        $("#output-field").val(JSON.stringify(output));
        $("#output-form").submit();
    }
"""%(entry_key, 'FractionLameller', entry['FractionLameller'])
)
compute_button.callback = button_callback
submit_button.callback = button_callback

### Mark areas that appear to be lameller
In U-Nb, lameller decomposition is often nucleated at grain boundaries and inclusions (aka "clunkers"). Consequently, the most likely regions to see decomposition are near triple points and small grains (which could be near the intersection point of 4 different grains [3D])

Under light microscopy, these regions typically appear as dark regions. So, look for dark areas near grain boundaries.

In [17]:
layout = column(
    row(widgetbox(mark_type), widgetbox(brush_type)),
    plot,
    widgetbox(results_box),
    row(widgetbox(submit_button), widgetbox(compute_button)),
    widgetbox(notes_box, sizing_mode='scale_width'),
)
show(layout)

### Package Output
Put the stuff into the `pinyon` variable

In [13]:
pinyon = {}
pinyon['scripts'], pinyon['div'] = components(layout)

## Output Cell (programmatically generated)

These commands connect to the Pwerwall server and send all desired output results back to the server

In [14]:
# pickle.dumps(pinyon)